텍스트 분석 기초
=========

텍스트 데이터 정제에서 가장 기본적으로 사용되는 R 코드를 정리하여 공유합니다. 본 코드는 영어 텍스트 데이터 정제를 기준으로 작성되었습니다.


In [3]:
#### Text review analysis ####
# github commit: http://datum.io/rrstudiogithub-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0/
# from https://rpubs.com/pjmurphy/265713

#리뷰할 텍스트가 들은 컬럼 추출 
review <-subset(review, Recommend<=7)  
text <- review[c(20)]

#CSV 추출하여 데이터 저장 
write.csv(text,"text.csv")        # csv extraction (CSV 추출)
#write.table(text,"text.txt", sep="\t")        # text extraction (텍스트 파일 csv로 추출)


#Install packages (관련 패키지 설치) 
install.packages("tm")          # text analysis package
install.packages("SnowballC")
install.packages("stringr")

텍스트 데이터를 불러오기 위한 라이브러리 오픈, 텍스트 데이터 불러오기, 데이터 확인 

In [ ]:
#### loading the text #### 
cname<- file.path("~", "directory", "medallia")
cname 
dir(cname)
length(dir(cname))       #from: http://antilibrary.org/490 #텍스트 파일의 길이 확인 

library(tm)
docs <- VCorpus(DirSource(cname))      #VCorpus로 전환 


#Check the text type and info (텍스트 타입과 그 외 정보 확인)
docs
class(docs)
class(docs[[1]])
summary(docs)
inspect(docs)
inspect(docs[1])     # (docs[1]) = docs3이라는 뭉치에 속한 1번 텍스트 파일 확인  
                     # (docs) = docs3 전체 뭉치 확인 

writeLines(as.character(docs[1]))      # writelines = 분석결과 확인 

텍스트 데이터 정제
------------- 

* 텍스트 데이터에서 구두점 및 특수기호 삭제
* 

In [ ]:
#### data cleansing ####

#Removing punctuation(구두점)
library(tm)
docs <- tm_map(docs,removePunctuation)   # 구두점 삭제 
writeLines(as.character(docs[2])) # Check to see if it worked.
                                  # The 'writeLines()' function is commented out to save space.
                                  # removePunctuation uses gsub('[[:punct:]]','',x) i.e. removes symbols: !"#$%&'()*+, \-./:;<=>?@[\\\]^_{|}~`. To remove other symbols, like typographic quotes or bullet signs (or any other), declare your own transformation function:
                                  # Or you can go further and remove everything that is not alphanumerical symbol or space:

removeSpecialChars <- function(x) gsub("[^a-zA-Z0-9 ]","",x)     # removeSpecialChars 함수 정의 
docs <- tm_map(docs, removeSpecialChars)     # removeSpecialChars 적용 


#ascii character 삭제 
for (j in seq(docs)) {
    docs[[j]] <- gsub("\\.", " ", docs[[j]])        # 온점 제거 
    docs[[j]] <- gsub("\\,", " ", docs[[j]])        # 반점 제거 
    docs[[j]] <- gsub(""\\d+"", " ", docs[[j]])     # 숫자 제거 
    docs[[j]] <- gsub("-", " ", docs[[j]]) }        # 하이픈(-) 제거 

#docs <- tm_map(docs, removeNumbers)    # 숫자 삭제 
#writeLines(as.character(docs[1]))      # Check it if the above codes are worked. 

docs <- tm_map(docs, tolower)     # 영어 대문자에서 소문자로 변환 
DocsCopy <- docs                  # raw data 복사  

텍스트 데이터 정제 
----------------

* 불필요한 단어 삭제 (be동사 및 기타 단어) = stopwords 목록을 통해 제거
    * stopwords 목록은 분석하는 사람이 직접 커스터마이징 할 수 있음 
* gsub을 활용한 특정 단어 패턴 수정 및 오타 수정  

In [ ]:

# For a list of the stopwords(common words) that usually have no analytic value. 
# see: length(stopwords("english"))   

#stopwords("english")   
docs <- tm_map(docs, removeWords, stopwords("english"))   
docs <- tm_map(docs, PlainTextDocument)


#### Data Cleaning ####

#단어 삭제 

#Removing particular word(단어 삭제)
docs <- tm_map(docs, removeWords, c("word1", "word2"))      # c 뒤의 "word1", "word2"에 지우고 싶은 단어 넣기

#Replacing particular word(단어 대체)
#sub(pattern, replacement, docs)     # sub = 텍스트에 등장하는 맨 첫번째 패턴 단어만 바꿈  
#gsub(pattern, replacement, docs)    # gsub = 텍스트에 등장하는 모든 패턴 단어를 바꿈 
                                     # Ref: http://rfunction.com/archives/2354 

for (j in seq(docs))
{
  docs[[j]] <- gsub("not answered", "not_answered", docs[[j]])     # pattern, replacement, dataname 순서로 작성 
  docs[[j]] <- gsub("not solved", "not_solved", docs[[j]])
  docs[[j]] <- gsub("qpp", "app", docs[[j]])  }     # 상황에 따라gsub 통해서 오타 교정(그러나 비추천)


텍스트 정리 후 정제 마무리를 위한 마지막 과정
* stemming 
* 공백 제거 
* txt로 변환 

In [ ]:

#Removing common word endings (e.g., “ing”, “es”, “s”)
#docs_st<-tm_map(docs, stemDocument)     # 가끔 stemDocument는 동작을 안함. 비추천. 
docs_st <- tm_map(docs, stemming = TRUE, language="english")   
docs_st <- tm_map(docs, PlainTextDocument)

docs2 <- tm_map(docs, stripWhitespace)      # 텍스트의 공백 삭제 
#writeLines(as.character(docs[1]))          # Check to see if it worked.

# plain text로 변환하기 전에 정제 작업을 모두 마무리 하였는지 확인 
docs2 <- tm_map(docs2, PlainTextDocument)

dtm2 <- DocumentTermMatrix(docs2)         # Create a document term matrix.
dtm   
inspect(dtm2)             # dtm으로 확인 가능한 것: Non-/sparse entries, Sparsity, Maximal term length, Weighting 

tdm2 <- TermDocumentMatrix(docs2)        # Term Document Matrix 변환 
tdm

freq2 <- colSums(as.matrix(dtm2))   # Organize terms by their frequency
length(freq2)   
#freq2                              # freq 목록 확인 
ord <- order(freq2)   

findFreqTerms(dtm2, lowfreq=50)            #the word list over than freq>50
findFreqTerms(dtm2, lowfreq=100)           #the word list over than freq>100

#sparse terms 제거   
#The ‘removeSparseTerms()’ 는 텍스트 내에서 자주 사용되지 않는 단어를 제거함. corpus 내에는 많이 언급되는 단어만 남게 됨. 상황에 따라 다를 수 있지만 20%가 적절해보임. 
#This makes a matrix that is 20% empty space, maximum.   
dtms2 <- removeSparseTerms(dtm2, 0.2) 
dtms2

#Frequency 단어 데이터 추출 
wf <- data.frame(word=names(word), freq2=freq)      # frequency 단어 테이블 추출 
write.csv(wf, "word_freq.csv")